In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
from scipy.optimize import minimize
from random import choices, randrange
import time
import matplotlib. pyplot as plt

In [2]:
def deconvo(A, b , solv):
    
    bounds = []
    for i in range(len(A.columns)):
        bounds.append([0., None])
        
    start_time = time.time()
    
    if solv == "nnls":
        x, rnorm = nnls(A,b)
        # Applaying sum-to-one and converting to prct:
        x = x/np.sum(x) * 100
        
    elif solv == "SLSQP" or solv == "trust-constr": 
        #Use nnls to get initial guess
        x0, rnorm = nnls(A,b)
        #Define minimisation function
        def fn(x, A, b):
            return np.linalg.norm(A.dot(x) - b)

        #Define constraints and bounds
        cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
        
        #Call minimisation subject to these values
        if solv == "trust-constr":
            minout = minimize(fn, x0, args=(A, b), method='trust-constr',bounds=bounds,constraints=cons)
        else:
            minout = minimize(fn, x0, args=(A, b), method='SLSQP',bounds=bounds,constraints=cons)
    
        x = minout.x
    else:
        raise ValueError("Unkown solver")  
    
    endtime = time.time()
    elapsed_time = (endtime - start_time)
    
    return x, elapsed_time

In [3]:
def rand_bulk(meta, raw_sc_data, kcell, rand_index = None):
    # random selection of kcell index
    if rand_index == None:
        rand_index = choices(range(len(meta.iloc[:, 0])), k = kcell)
    
    
    # obtention of selected cell names
    rand_cells = meta.iloc[rand_index,0]
    
    # subsetting of the kcell cells
    rand_expr = raw_sc_data.loc[:,rand_cells]
    
    # Suming values
    sum_rand_expr = rand_expr.sum(axis=1)
    
    # Rematching with gene names 
    gene_names = raw_sc_data.iloc[:, 0]
    simu_bulk = pd.concat([gene_names, sum_rand_expr], axis=1)
    
    ## Calculating true prop
    # obtention of selected cell clusters
    rand_clust = meta.iloc[rand_index,7]
    # Summing values
    clust_sum = rand_clust.value_counts()
    # Obtaining proportions
    clust_true_prop = (clust_sum/kcell) * 100

    return simu_bulk, clust_true_prop

In [4]:
def intersect(simu_bulk, sc_avg_data):
    ## Adjusting sc_avg_data and simulated bulk
    # separating into gene names and values 
    sc_avg_gene = sc_avg_data.iloc[:, 0]
    sc_avg_expr = sc_avg_data.iloc[:,1:]
    # subsetting simu bulk data
    sub_simu_bulk = simu_bulk[simu_bulk.iloc[:,0].isin(sc_avg_gene)]
    
    return sub_simu_bulk.iloc[:,1], sc_avg_expr

In [5]:
def rand_subset(meta, sc_raw_data, kcell, pops):
    
    # calculating proportion for each pop
    parts = len(pops)
    goal = kcell
    prop = []
    
    for i in range(parts - 1):
        selected_numb = randrange(goal)
        prop.append(selected_numb)
        goal = goal - selected_numb 
    prop.append(goal)
    
    # selecting those cells
    i = 0
    selected_cells_index = []
    for pop in pops:
        cell_of_pop = meta[meta['seurat_clusters'] == pop].iloc[:, 0]
        selected_cells_index = selected_cells_index + (choices(cell_of_pop.index, k = prop[i]))
        i = i+1

    simu_bulk, true_prop = rand_bulk(meta, sc_raw_data, kcell, selected_cells_index)
    return simu_bulk, true_prop

In [6]:
single_cell = pd.read_csv('Selected_gene_sc.tsv', sep = '\t')
single_cell

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.323555,0.094705,0.192635,0.577982,0.353719,0.128162,0.032882,0.104255,0.600437,0.131016,0.029491,0.132231,0.017921,0.362637,0.036290,0.015385,0.180328,0.076923,0.304348
1,0.000863,0.003055,0.000000,0.000000,0.001653,0.005059,0.001934,0.000000,0.000000,0.005348,0.000000,0.002755,0.853047,0.000000,0.274194,0.558974,0.032787,0.329670,0.000000
2,0.005177,0.006110,0.001416,0.009174,0.009917,0.008432,0.174081,0.002128,0.008734,0.002674,0.439678,0.002755,0.465950,0.014652,0.411290,0.358974,0.038251,0.153846,0.000000
3,0.153581,0.068228,0.264873,0.397554,0.171901,0.178752,0.166344,0.287234,0.257642,0.697861,0.083110,0.228650,0.111111,0.183150,0.032258,0.071795,0.180328,0.087912,0.130435
4,0.006903,1.009165,0.000000,0.000000,0.004959,0.005059,0.000000,0.000000,0.002183,0.002674,0.000000,0.000000,0.007168,0.000000,0.000000,0.000000,0.038251,0.000000,0.065217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,0.993097,0.945010,0.603399,0.749235,0.662810,0.598651,0.843327,0.525532,0.772926,0.548128,1.428954,0.586777,0.544803,0.501832,0.584677,0.707692,0.628415,0.802198,0.782609
2860,0.260569,0.423625,0.120397,0.152905,0.180165,0.139966,0.396518,0.140426,0.213974,0.125668,0.605898,0.195592,0.394265,0.139194,0.334677,0.205128,0.469945,0.164835,0.347826
2861,0.414150,0.120163,0.140227,0.501529,0.198347,0.124789,0.243714,0.306383,0.879913,0.141711,0.233244,0.088154,0.215054,0.461538,0.318548,0.128205,0.229508,0.197802,0.369565
2862,0.053494,0.177189,0.021246,0.079511,0.071074,0.057336,0.334623,0.072340,0.019651,0.136364,0.378016,0.066116,0.136201,0.051282,0.282258,0.076923,0.852459,0.109890,0.043478


In [7]:
bulk = pd.read_csv('Selected_gene_bulk.tsv', sep = '\t')
gene = bulk['id']
bulk_values = bulk.iloc[:,1:]
bulk_values

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110
0,0.272364,0.428677,0.108904,0.282876,0.443393,0.057185,0.277305,0.142332,0.119310,0.257542,...,0.297698,0.227899,0.058972,0.263744,0.202460,0.269000,0.340271,0.127299,0.322611,0.120151
1,0.651938,1.162660,0.593676,0.925709,1.217457,2.514403,1.598007,0.746156,0.639809,0.927875,...,1.868096,0.907949,2.120641,1.265974,2.091835,0.367555,0.927659,1.486896,0.618150,0.629630
2,0.014840,0.020105,0.031594,0.007659,0.001436,0.009095,0.003830,0.029679,0.008617,0.003351,...,0.012446,0.011010,0.359023,0.006223,0.013882,0.008617,0.006702,0.032073,0.005266,0.006702
3,1.094460,1.051771,0.673025,1.704511,0.238874,0.592189,0.517711,1.271269,0.511656,0.376627,...,0.290039,0.355132,0.717227,0.418105,0.392371,0.585226,0.660914,0.913412,1.101726,1.521344
4,0.251093,0.134916,0.502811,0.120550,1.321674,0.080575,0.194878,0.446596,0.178014,1.098064,...,0.108682,0.073704,0.291068,0.623985,0.090568,0.236727,0.069332,0.147408,0.049344,0.047470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,0.291871,0.210932,1.013315,0.321654,0.297828,0.261388,0.401191,0.625088,0.331114,0.351086,...,0.254730,0.332165,0.925368,0.489488,0.287316,0.257884,0.322705,0.389278,0.343378,0.317449
2860,0.021818,0.025455,0.092468,0.021818,0.015584,0.072727,0.021818,0.121039,0.018701,0.017143,...,0.028052,0.011429,0.339221,0.017143,0.012987,0.020260,0.005714,0.058182,0.018701,0.025455
2861,0.084702,0.059421,0.127269,0.057044,0.015774,0.050562,0.047537,0.124244,0.058773,0.029602,...,0.052723,0.031547,0.173509,0.059637,0.058557,0.064823,0.053803,0.103933,0.076707,0.095938
2862,0.036061,0.021267,0.052011,0.023347,0.010402,0.021960,0.020342,0.081600,0.022654,0.014101,...,0.031438,0.018724,0.522191,0.033287,0.033518,0.032825,0.027739,0.044152,0.018955,0.023810


In [6]:
meta = pd.read_csv('dataset/meta_data_h2', sep = '\t')
meta

,Unnamed: 0,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,SCT_snn_res.0.8,seurat_clusters
0,human2_lib1.final_cell_0001,human2,9323,2267,5641,2044,5,5
1,human2_lib1.final_cell_0002,human2,6732,1632,5399,1627,5,5
2,human2_lib1.final_cell_0003,human2,10762,2815,5738,2357,0,0
3,human2_lib1.final_cell_0004,human2,8546,2366,5611,2264,5,5
4,human2_lib1.final_cell_0005,human2,8651,2331,5607,2204,5,5
...,...,...,...,...,...,...,...,...
1719,human2_lib3.final_cell_0561,human2,1571,933,3696,1073,7,7
1720,human2_lib3.final_cell_0562,human2,1703,827,3891,936,10,10
1721,human2_lib3.final_cell_0563,human2,1447,859,3639,1031,6,6
1722,human2_lib3.final_cell_0564,human2,1514,800,3734,955,10,10


In [9]:
single_cell_h2 = pd.read_csv('dataset/selected_genes_sc_h2', sep = '\t')

In [7]:
sc_raw = pd.read_csv('dataset/treated_h2_data', sep = '\t')
full_gene = sc_raw['Unnamed: 0']
sc_raw

,Unnamed: 0,human2_lib1.final_cell_0001,human2_lib1.final_cell_0002,human2_lib1.final_cell_0003,human2_lib1.final_cell_0004,human2_lib1.final_cell_0005,human2_lib1.final_cell_0006,human2_lib1.final_cell_0007,human2_lib1.final_cell_0008,human2_lib1.final_cell_0009,...,human2_lib3.final_cell_0556,human2_lib3.final_cell_0557,human2_lib3.final_cell_0558,human2_lib3.final_cell_0559,human2_lib3.final_cell_0560,human2_lib3.final_cell_0561,human2_lib3.final_cell_0562,human2_lib3.final_cell_0563,human2_lib3.final_cell_0564,human2_lib3.final_cell_0565
0,A1BG,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,1.098612,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,A1CF,0.000000,1.098612,0.693147,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,...,0.0,0.0,1.098612,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,A2M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.098612,0.000000,0.0,0.000000,0.0
3,A4GALT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.098612,0.0
4,AAAS,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14192,ZYG11B,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0
14193,ZYX,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,...,0.0,0.0,0.000000,1.098612,0.0,0.000000,1.098612,0.0,1.098612,0.0
14194,ZZEF1,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.098612,0.000000,0.0,0.000000,0.0
14195,ZZZ3,0.000000,1.098612,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [33]:
clust_types = [[0,1,11],[2,3],[6],[9,4],[5],[8],[12],[7],[10]]

tested_clust = []
results = {}
for types in clust_types:
    for clust in types:
        for types2 in clust_types:
            for clust2 in types2:
                if clust != clust2:
                    tested_clust = [clust,clust2]
                    simu_bulk, true_prop = rand_subset(meta, sc_raw, 1000, tested_clust)
                    b, A = intersect(simu_bulk, single_cell_h2)
                    res, comp_time = deconvo(A, b, 'nnls')
                    results[tuple(tested_clust)] = [res, true_prop]
                
        

In [22]:
def print_res(res):
    for i in range(len(res)):
        print("cluster "+str(i)+": "+str(res[i])+"%")

In [34]:
for comb in results:
    print_res(results[comb][0])
    print(results[comb][1])

cluster 0: 0.0%
cluster 1: 4.96197569374512%
cluster 2: 0.0%
cluster 3: 12.61829080774835%
cluster 4: 24.503661742982167%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.331810547609428%
cluster 8: 0.9628477116087079%
cluster 9: 20.094739488969022%
cluster 10: 11.031917357547723%
cluster 11: 0.0%
cluster 12: 24.49475664978948%
0    97.5
1     2.5
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 5.177627812399114%
cluster 2: 0.0%
cluster 3: 12.786393325366047%
cluster 4: 23.417501337845785%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.3390868016004074%
cluster 8: 1.0096537130986256%
cluster 9: 21.055838205666007%
cluster 10: 10.266306424744325%
cluster 11: 0.0%
cluster 12: 24.947592379279683%
0     84.5
11    15.5
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 4.811829366743043%
cluster 2: 0.0%
cluster 3: 12.797994058075155%
cluster 4: 24.202896460222608%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.3373226003003593%
cluster 8: 0.9391647413771106%
cluster 

Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 2.4140182464621276%
cluster 2: 0.0%
cluster 3: 9.031842161188015%
cluster 4: 22.04327793606925%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 14.561233072042876%
cluster 8: 0.6840604765435847%
cluster 9: 14.367954437362258%
cluster 10: 7.101046955317362%
cluster 11: 0.0%
cluster 12: 29.796566715014528%
1    63.1
7    36.9
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 1.558819215700888%
cluster 2: 0.0%
cluster 3: 6.490303931299891%
cluster 4: 17.628981824543512%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 23.538579453753066%
cluster 8: 0.5863445274693091%
cluster 9: 14.351231863761827%
cluster 10: 2.5037193902215886%
cluster 11: 0.0%
cluster 12: 33.342019793249925%
7     64.5
11    35.5
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 0.5641373613946514%
cluster 2: 8.173476167955398%
cluster 3: 12.431428628457558%
cluster 4: 13.774272890516315%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 10.27

In [38]:
tested_clust = []
results = {}
for types in clust_types:
    tested_clust = types
    simu_bulk, true_prop = rand_subset(meta, sc_raw, 1000, tested_clust)
    b, A = intersect(simu_bulk, single_cell_h2)
    res, comp_time = deconvo(A, b, 'nnls')
    results[tuple(tested_clust)] = [res, true_prop]
                

In [39]:
for types in results:
    print_res(results[types][0])
    print(results[types][1])

cluster 0: 0.0%
cluster 1: 5.615542450348123%
cluster 2: 0.0%
cluster 3: 13.849164227666225%
cluster 4: 20.501294690147727%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.5599890095636693%
cluster 8: 1.218004254282307%
cluster 9: 22.44393422179815%
cluster 10: 8.427727683233988%
cluster 11: 0.0%
cluster 12: 26.384343462959812%
11    57.0
1     26.9
0     16.1
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 0.954475937289629%
cluster 2: 0.0%
cluster 3: 27.47138853232018%
cluster 4: 13.292845868661459%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.839988325839016%
cluster 8: 0.8152593948201454%
cluster 9: 29.139040295229528%
cluster 10: 3.4047406104012934%
cluster 11: 0.0%
cluster 12: 23.082261035438744%
2    65.0
3    35.0
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 0.0%
cluster 2: 0.0%
cluster 3: 1.5352018746649971%
cluster 4: 50.52701362305918%
cluster 5: 0.0%
cluster 6: 13.861763022312095%
cluster 7: 0.39816187459241037%
cluster 8: 0.095459935276072

In [41]:
tested_clust = []
results = {}
for types in clust_types:
    for clust in types:
        tested_clust = [clust]
        simu_bulk, true_prop = rand_subset(meta, sc_raw, 1000, tested_clust)
        b, A = intersect(simu_bulk, single_cell_h2)
        res, comp_time = deconvo(A, b, 'nnls')
        results[tuple(tested_clust)] = [res, true_prop]
                

In [42]:
for clust in results:
    print_res(results[clust][0])
    print(results[clust][1])

cluster 0: 0.0%
cluster 1: 4.888034280924112%
cluster 2: 0.0%
cluster 3: 12.586611690182192%
cluster 4: 24.838692650200837%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.2720120181536338%
cluster 8: 0.9513453234489603%
cluster 9: 19.98473486976728%
cluster 10: 11.180712568588518%
cluster 11: 0.0%
cluster 12: 24.297856598734466%
0    100.0
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 4.750685351192657%
cluster 2: 0.0%
cluster 3: 13.417216131577266%
cluster 4: 24.80261361750499%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.3808986151659428%
cluster 8: 1.0080697837559347%
cluster 9: 18.280265108505734%
cluster 10: 11.310409304396956%
cluster 11: 0.0%
cluster 12: 25.049842087900508%
1    100.0
Name: seurat_clusters, dtype: float64
cluster 0: 0.0%
cluster 1: 6.3231358400435855%
cluster 2: 0.0%
cluster 3: 14.16142103144077%
cluster 4: 17.634664544378285%
cluster 5: 0.0%
cluster 6: 0.0%
cluster 7: 1.751980571790796%
cluster 8: 1.398334031795868%
cluster 9: 25.08476816152992

In [79]:
top5 = pd.read_csv('dataset/top5_genes_h2.tsv', sep = '\t')
top5

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.144330,0.128920,0.035176,0.017442,0.022901,0.023622,0.245098,0.484848,0.011111,0.013158,18.765625,0.047619,0.000000
1,0.652921,0.501742,0.155779,0.343023,0.564885,1.944882,0.892157,0.121212,12.655556,0.105263,0.546875,0.269841,0.043478
2,1.171821,0.891986,4.462312,10.302326,0.076336,2.354331,0.068627,0.141414,0.344444,0.092105,0.250000,0.603175,0.260870
3,0.567010,0.550523,0.045226,0.116279,6.465649,0.212598,4.656863,0.050505,0.388889,2.131579,14.250000,0.412698,0.173913
4,2.027491,3.010453,18.065327,6.366279,0.893130,1.393701,0.284314,0.747475,0.988889,0.710526,1.515625,1.317460,0.869565
5,30.635739,29.592334,2.728643,3.343023,6.106870,5.677165,3.960784,0.454545,15.077778,3.934211,10.078125,21.031746,0.260870
6,0.061856,0.073171,0.075377,0.122093,0.458015,0.086614,0.490196,140.070707,0.033333,0.223684,0.656250,0.063492,0.086957
7,0.034364,0.034843,0.005025,0.034884,0.000000,0.031496,0.009804,41.333333,0.033333,0.144737,0.640625,0.000000,0.217391
8,8.934708,13.174216,17.809045,17.906977,1.213740,14.818898,0.225490,1.101010,9.633333,0.710526,4.109375,26.238095,0.478261
9,0.020619,0.003484,0.005025,0.005814,0.900763,0.000000,0.098039,0.656566,0.000000,6.828947,0.046875,0.000000,0.000000


In [68]:
top10 = pd.read_csv('dataset/top10_genes_h2.tsv', sep = '\t')
top10

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.484536,2.689895,7.085427,13.436047,0.213740,2.606299,0.117647,0.333333,3.133333,0.157895,1.265625,2.031746,0.130435
1,0.144330,0.128920,0.035176,0.017442,0.022901,0.023622,0.245098,0.484848,0.011111,0.013158,18.765625,0.047619,0.000000
2,0.013746,0.013937,3.547739,1.773256,0.022901,0.007874,0.009804,0.030303,0.277778,0.026316,0.031250,0.015873,0.000000
3,3.608247,5.853659,0.346734,1.000000,2.114504,0.086614,0.166667,0.272727,0.177778,0.605263,1.437500,0.444444,0.000000
4,1.017182,0.540070,1.502513,1.924419,15.984733,0.062992,5.235294,1.737374,0.255556,6.697368,1.781250,1.428571,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.034364,0.146341,0.040201,0.075581,0.595420,0.023622,0.745098,0.282828,0.366667,0.368421,0.203125,0.380952,13.086957
96,0.106529,0.452962,0.929648,5.604651,0.610687,0.267717,1.058824,0.030303,0.144444,0.092105,0.312500,0.253968,0.086957
97,230.089347,282.268293,20.638191,16.755814,4.755725,8.952756,0.852941,2.727273,37.933333,2.342105,78.656250,255.761905,1.782609
98,0.000000,0.003484,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.265625,0.000000,0.000000


In [69]:
top20 = pd.read_csv('dataset/top20_genes_h2.tsv', sep = '\t')

In [70]:
top50 = pd.read_csv('dataset/top50_genes_h2.tsv', sep = '\t')

In [77]:
top5_bulk = pd.read_csv('dataset/top5_h2_genes_bulk.tsv', sep='\t')
top5_gene = top5_bulk.iloc[:,0]
top5_bulk = top5_bulk.iloc[:,1:]

In [72]:
top10_bulk = pd.read_csv('dataset/top10_h2_genes_bulk.tsv', sep='\t')
top10_gene = top10_bulk.iloc[:,0]
top10_bulk = top10_bulk.iloc[:,1:]

In [73]:
top20_bulk = pd.read_csv('dataset/top20_h2_genes_bulk.tsv', sep='\t')
top20_gene = top20_bulk.iloc[:,0]
top20_bulk = top20_bulk.iloc[:,1:]


In [74]:
top50_bulk = pd.read_csv('dataset/top50_h2_genes_bulk.tsv', sep='\t')
top50_gene = top50_bulk.iloc[:,0]
top50_bulk = top50_bulk.iloc[:,1:]


In [80]:
sc_top5 = sc_raw[sc_raw['Unnamed: 0'].isin(top5_gene)]
meta_top5= meta[meta['']]

In [84]:
simu_bulk_top5, true_prop = rand_bulk(meta, sc_top5, 1000)
simu_bulk_top5

,Unnamed: 0,0
144,ACP5,111.763320
700,AQP3,363.843309
1500,C1QL1,672.145655
2079,CD74,368.073864
2181,CDKN1C,820.163876
2529,CLU,2001.410759
2628,COL1A1,340.473327
2629,COL1A2,228.425431
2733,CPE,1916.173665
2823,CRYAB,160.874681


In [85]:
res, comp_time = deconvo(top5, simu_bulk_top5.iloc[:,1], 'nnls')

In [86]:
res

array([ 0.        ,  0.        ,  0.        , 10.70823528, 43.18000281,
        0.56989103,  0.45863736,  3.00063906,  1.19845861,  7.03452109,
       29.29086568,  0.        ,  4.55874907])

In [87]:
true_prop

0     17.0
1     15.8
2     12.0
3     10.1
4      7.0
5      6.9
7      6.4
8      5.9
6      5.5
9      5.0
10     3.9
11     3.1
12     1.4
Name: seurat_clusters, dtype: float64

In [8]:
top1 = pd.read_csv('dataset/top1_genes_h2.tsv', sep = '\t')

In [36]:
top1_bulk = pd.read_csv('dataset/top1_h2_genes_bulk.tsv', sep='\t')
top1_gene = top1_bulk.iloc[:,0]
top1_bulk_val = top1_bulk.iloc[:,1:]
top1_bulk_val

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110
0,0.231538,0.319822,0.305386,0.268184,0.031649,0.041644,0.168240,0.200999,0.087174,0.147696,...,0.203776,0.109939,0.090505,0.270405,0.125486,0.113270,0.109384,0.453082,0.151582,0.081066
1,2.568414,4.426140,3.237066,3.012253,0.084922,0.459837,0.345473,8.001191,0.653336,0.371511,...,1.077774,1.632743,13.278761,0.315861,1.794588,1.931416,1.716133,2.100919,1.550545,1.825732
2,926.439158,3229.354134,919.164587,956.267551,686.764431,915.067083,1193.736349,449.166927,497.027301,745.290952,...,966.640406,2061.443838,250.529641,680.063963,600.128705,1765.579563,2146.549142,754.205928,3173.847114,1683.893136
3,0.227045,0.137730,1.814691,0.219533,0.153589,0.025042,0.075960,0.601836,0.159432,0.165275,...,0.704508,0.131886,15.279633,0.982471,0.166110,0.171119,0.081803,0.254591,0.027546,0.080134
4,32.062848,182.419665,131.458186,62.998986,216.687278,33.005575,228.061328,160.973137,58.978713,142.791181,...,13.556513,230.930563,1.822098,12.414090,47.105930,134.145464,39.905727,4.887988,125.732387,12.895590
5,2.194399,7.899231,2.012630,0.982702,0.904448,0.653213,3.254256,2.588138,0.615596,1.031027,...,0.607633,2.140857,1.461010,0.271829,0.556562,2.205382,1.899231,1.348710,2.707853,3.973641
6,2.875576,8.891841,3.632963,4.459474,3.349146,6.872323,11.533207,4.153158,1.836812,3.553538,...,3.456763,6.602060,1.331255,3.521008,3.277040,2.915695,3.451884,6.108430,3.666847,1.621578
7,0.899559,1.653744,0.902643,0.987225,0.110573,1.501762,0.602203,3.172247,0.167841,0.459471,...,0.283260,0.450661,6.322907,0.080617,1.285022,1.288987,0.648458,2.991189,0.940969,0.825991
8,19.408389,12.136865,7.909492,210.549669,5.487859,8.944812,5.799117,49.986755,10.075055,0.406181,...,2.392936,10.094923,102.969095,1.293598,13.364238,20.607064,5.258278,100.388521,211.192053,53.554084
9,503.896509,238.493766,908.617207,269.281796,1108.492519,40.508728,440.231920,361.007481,486.709476,925.254364,...,1136.538653,303.553616,663.408978,1286.779302,369.983791,453.704489,619.221945,210.487531,73.602244,58.201995


In [55]:
sc_top1 = sc_raw[sc_raw['Unnamed: 0'].isin(top1_gene)]
simu_bulk_top1, true_prop = rand_bulk(meta, sc_top1, 10000)

In [30]:
simu_bulk_top1

,Unnamed: 0,0
144,ACP5,956.804403
2628,COL1A1,3460.742900
4715,GCG,32783.529893
4736,GDF15,6263.588866
5592,IAPP,15436.469022
6530,LOXL4,4144.856262
8576,PDK4,6015.793332
8940,PLVAP,574.014172
9227,PPY,6084.520941
9895,REG1A,4545.827020


In [12]:
top1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.144330,0.128920,0.035176,0.017442,0.022901,0.023622,0.245098,0.484848,0.011111,0.013158,18.765625,0.047619,0.000000
1,0.061856,0.073171,0.075377,0.122093,0.458015,0.086614,0.490196,140.070707,0.033333,0.223684,0.656250,0.063492,0.086957
2,711.625430,574.435540,6.291457,5.581395,19.091603,6.622047,2.960784,12.303030,7.066667,11.039474,128.343750,722.698413,3.434783
3,0.412371,0.341463,0.095477,0.087209,13.366412,0.181102,25.852941,5.121212,0.088889,37.855263,5.296875,0.111111,0.521739
4,1.676976,1.787456,189.432161,190.389535,2.198473,1.622047,0.676471,2.898990,2.233333,0.894737,6.187500,0.936508,0.826087
5,1.742268,7.296167,0.015075,0.127907,0.297710,0.015748,0.029412,0.050505,0.211111,0.131579,2.500000,0.365079,0.043478
6,1.247423,3.742160,0.542714,1.453488,0.633588,1.070866,0.382353,0.919192,2.922222,0.434211,1.343750,8.555556,0.086957
7,0.000000,0.000000,0.010050,0.000000,0.000000,0.015748,0.000000,0.000000,0.000000,0.000000,0.203125,0.015873,46.695652
8,0.573883,0.752613,4.195980,0.436047,0.366412,0.496063,0.225490,0.454545,736.244444,0.644737,2.515625,3.603175,0.260870
9,0.140893,0.324042,0.301508,0.232558,1.664122,0.259843,231.725490,0.161616,0.244444,0.250000,16.531250,0.222222,0.173913


In [61]:
res, comp_time = deconvo(top1, simu_bulk_top1.iloc[:,1], 'nnls')

In [62]:
res

array([ 0.        ,  7.692348  ,  0.        , 15.66507077, 25.815528  ,
        1.44642793,  2.89921818,  4.91545165,  1.73819511, 23.59374606,
       13.66351757,  0.        ,  2.57049674])

In [63]:
for i in range(len(res)):
    print("cluster "+str(i)+": "+str(res[i])+"%")
print(true_prop)

cluster 0: 0.0%
cluster 1: 7.692348002245633%
cluster 2: 0.0%
cluster 3: 15.665070773637726%
cluster 4: 25.815527995449806%
cluster 5: 1.446427929377473%
cluster 6: 2.899218181717322%
cluster 7: 4.915451645422506%
cluster 8: 1.7381951079063385%
cluster 9: 23.593746055154092%
cluster 10: 13.663517567103323%
cluster 11: 0.0%
cluster 12: 2.570496741985789%
1     17.52
0     16.89
2     11.33
3      9.46
4      7.96
5      7.12
6      5.85
7      5.66
8      5.24
9      4.54
10     3.57
11     3.53
12     1.33
Name: seurat_clusters, dtype: float64


In [93]:
true_prop

0     18.1
1     16.1
2     12.9
3     10.9
4      7.4
5      5.8
6      5.5
9      5.3
8      5.0
7      5.0
10     3.8
11     2.9
12     1.3
Name: seurat_clusters, dtype: float64

In [108]:
res_other, comp_time = deconvo(top1, simu_bulk_top1.iloc[:,1], 'SLSQP')

In [109]:
res_other/np.sum(res_other) * 100

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [13]:
maxes = top1.idxmax(axis = 1)
maxed = pd.DataFrame(maxes)
maxed.at[2] = 2
maxed.iloc[:,0]

0     10
1      7
2      2
3      9
4      3
5      1
6     11
7     12
8      8
9      6
10     4
11     5
Name: 0, dtype: object

In [59]:
naive_prop = []
i = 0
for i in range(len(simu_bulk_top1.index)):
    tot = np.sum(top1.iloc[i,:])
    bulk_val = simu_bulk_top1.iloc[i,1]    
    cells = bulk_val / tot
    line_prop = []
    for elt in top1.iloc[i,:]:
        line_prop.append(cells * (elt/ tot*100))
    naive_prop.append(line_prop)
        
    i += 1
naive_prop

[[33.87629921410415,
  30.25934242892479,
  8.25628397931684,
  4.093855096056608,
  5.3751379887155455,
  5.544433673399499,
  57.52802912759283,
  113.80090125603823,
  2.6079373204508762,
  3.0883468268497203,
  4404.561640117737,
  11.176874230503751,
  0.0],
 [1.0164326538912924,
  1.2023654564323822,
  1.2386176812494394,
  2.006272593000545,
  7.526257055530943,
  1.423272495671928,
  8.055062698321516,
  2301.6877884116557,
  0.5477442634858631,
  3.675652294444608,
  10.783715187377931,
  1.043322406639739,
  1.428898078658773],
 [483.0054774273663,
  389.88982231394334,
  4.27023572254116,
  3.788291442688272,
  12.95814970168654,
  4.494618879328607,
  2.0095895700469733,
  8.350504041218668,
  4.796397887714761,
  7.49288324440048,
  87.11146576434786,
  490.52110473417144,
  2.3313090635529097],
 [34.026823317577445,
  28.175869551994023,
  7.8783210269215145,
  7.196079640563257,
  1102.9301259581257,
  14.94366984675498,
  2133.256644504858,
  422.57705654927827,
  7.334

In [60]:
df = pd.DataFrame.from_records(naive_prop)
moy_cell = df.mean()
res = (moy_cell / np.sum(moy_cell)) *100
res

0      6.976544
1     25.385779
2      3.502204
3      4.711745
4     10.285611
5      2.435467
6      5.012714
7      4.299002
8      4.991729
9      4.869581
10    14.215168
11    11.745059
12     1.569397
dtype: float64

In [58]:
true_prop

1     17.52
0     16.89
2     11.33
3      9.46
4      7.96
5      7.12
6      5.85
7      5.66
8      5.24
9      4.54
10     3.57
11     3.53
12     1.33
Name: seurat_clusters, dtype: float64

In [54]:
weight = true_prop/res
len(weight)

13